#**Import necessary libraries**

In [ ]:
#Importing Data manipulation and plotting modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import plot_importance
import seaborn as sns
import os

In [ ]:
#Importing libraries for Data pre-processing
from sklearn.preprocessing import StandardScaler as ss

In [ ]:
#Importing model for Dimentionality Reduction
from sklearn.decomposition import PCA

In [ ]:
#Importing libraries for performance measures
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc, roc_curve

In [ ]:
#Importing libraries For data splitting
from sklearn.model_selection import train_test_split

In [ ]:
#Importing libraries for Model pipelining
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

In [ ]:
#Importing libraries for model parameter search and hyperparameter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
#Importing Miscelaneous libraries
import time
import os
from scipy.stats import uniform

#**Loading dataset**

In [ ]:
#Display all rows
pd.set_option('display.max_columns', 100)

In [ ]:
os.chdir("../input") 
data = pd.read_csv("winequalityN.csv")

#**Get to know data and deleting rows having null values **

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.isnull().any()

In [ ]:
data.dropna(axis=0,inplace=True)

In [ ]:
data.isnull().any()

In [ ]:
data.shape

#**Exploring Dataset**

In [ ]:
g = sns.pairplot(data,palette="husl",diag_kind="kde",hue='type')

###**As we can see the figure shows many co-dependencies between features. Some observations -**
###**1. Negative correlation between - i)Fixed acidity & pH, ii)density & alcohol.**
###**2. No correlation - i)alcohol & pH, ii)alcohol &  fixed acidity, iii)alcohol & volatile acidity**
###**3. Positive correlation - i)density and fixed acidity, ii)density & residual sugar, iii)Citric acid & fixed acidity:initially no correlation; but gng forward positive correlation**
###**

In [ ]:
plt.figure(figsize=(12, 12))
sns.heatmap(data.corr(),annot=True,vmin=-1,cmap='YlGnBu')

###**Heatmap gives us more insights. Different shades of blue depict different levels of co-relationships between features.**
###**Positive values show linear or direct co-relationships between features. Higher the values stronger the co-relationships.**
###**They will be dark blue.**
###**On the other hand, negative values show an inverse co-relationship i.e as value of one feature increases the other one decreases.**
###**As the shade of blue decreases so does the co-relationship between the respective features.**
###**As you can see, the positive values show direct co-relationships between features.** 
###**The higher the values, the stronger these relationships are — they’ll be more bluish. **
###**That means, if one feature increases, the other one also tends to increase, and vice-versa.**
###**The squares that have negative values show an inverse co-relationship. **
###**The more negative these values get, the more inversely proportional they are, and they’ll be more blue. **
###**This means that if the value of one feature is higher, the value of the other one gets lower.**
###**Finally, values close to zero indicate almost no co-dependency between those sets of features.**

In [ ]:
plt.figure(figsize=(12,10))
sns.barplot(x=data.type, y=data.alcohol,data=data, hue=data.quality,palette='spring')
plt.xlabel('Type',fontsize=16)
plt.ylabel('Quality',fontsize=16)

###**Wine types are separated and each component measure is compared. We can observe that alcohol quantity is slightly more in white wine.**

In [ ]:
#Assuming wines with quality greater than 6 are best

wine_data = data.shape[0]
print("Total number of wine data: "+str(wine_data))
high_quality = data.loc[(data.quality)> 6]
print("Best quality wine entries: "+str(high_quality.shape[0]))
good_quality = data.loc[(data.quality)<5]
print("Good quality wine entries: "+str(good_quality.shape[0]))
end_of_month_wine = data.loc[(data.quality) < 3]
print("Mediocre quality wine entries: "+str(good_quality.shape[0]))

In [ ]:
table = data.pivot_table('alcohol',index='quality',columns='type')
q = table.quantile(0.90)
df = table[table < q]
plt.figure(figsize=(12,8))
plt.xticks(rotation=45)
ax = sns.boxplot(data=data)

In [ ]:
table_fa = data.pivot_table('fixed acidity',index='quality',columns='type',aggfunc='sum').sum(axis=1)
table_va = data.pivot_table('volatile acidity',index='quality',columns='type',aggfunc='sum').sum(axis=1)
table_ca = data.pivot_table('citric acid',index='quality',columns='type',aggfunc='sum').sum(axis=1)
table_rs = data.pivot_table('residual sugar',index='quality',columns='type',aggfunc='sum').sum(axis=1)
table_chl = data.pivot_table('chlorides',index='quality',columns='type',aggfunc='sum').sum(axis=1)
table_tsd = data.pivot_table('total sulfur dioxide',index='quality',columns='type',aggfunc='sum').sum(axis=1)
table_den = data.pivot_table('density',index='quality',columns='type',aggfunc='sum').sum(axis=1)
table_pH = data.pivot_table('pH',index='quality',columns='type',aggfunc='sum').sum(axis=1)
table_sul = data.pivot_table('sulphates',index='quality',columns='type',aggfunc='sum').sum(axis=1)
table_alcohol = data.pivot_table('alcohol',index='quality',columns='type',aggfunc='sum').sum(axis=1)
qual = table_alcohol.index.astype(int)
lbl = ['fa','va','ca','rs','chl','tsd','den','pH','sul','alcohol']

plt.figure(figsize=(12,14))
ax = sns.pointplot(x=qual, y=table_fa, color='y',scale=0.7)
ax = sns.pointplot(x=qual, y=table_va, color='b',scale=0.7)
ax = sns.pointplot(x=qual, y=table_ca, color='m', scale=1.0)
ax = sns.pointplot(x=qual, y=table_rs, color='thistle', scale=1.0)
ax = sns.pointplot(x=qual, y=table_chl, color='g', scale=1.0)
ax = sns.pointplot(x=qual, y=table_tsd, color='coral',scale=0.7)
ax = sns.pointplot(x=qual, y=table_den, color='olive', scale=1.0)
ax = sns.pointplot(x=qual, y=table_pH, color='sienna', scale=1.0)
ax = sns.pointplot(x=qual, y=table_sul, color='m', scale=1.0)
ax = sns.pointplot(x=qual, y=table_alcohol, color='springgreen', scale=1.0)

ax.set_xlabel(xlabel='Quality', fontsize=16)
ax.set_ylabel(ylabel='Quantity of each component', fontsize=16)
ax.legend(handles=ax.lines[::len(qual)+1],labels=lbl,fontsize=12)
plt.show();

###**We know that older the wine, better it tastes. Sulphur dioxide acts as a preservative and antioxidant which is widely used in winemaking.If it is carefully balanced out with pH then the quality of wine is preserved on storage. Hence we can observe that Total sulphur dioxide dominates all other wine components.It is followed by alcohol and density.**

In [ ]:
df = data.groupby(['quality'])
df_mean = df['fixed acidity','volatile acidity','citric acid'].aggregate(np.mean)
df_mean.plot(figsize=(10,8))
plt.xlabel('Quality',fontsize=16)
plt.ylabel('Quantity',fontsize=16)

###**The fixed acidity content is more as compared to volatile acid and citric acid. Combination of these acids contribute to the taste of wine.

In [ ]:
fig = plt.figure(figsize=(12, 10));
ax = fig.add_subplot(1,1,1)
bp = data.boxplot(grid=False,layout=(2,2),column=['fixed acidity','volatile acidity','citric acid','alcohol'], by=['quality'],ax=ax)


###**It can be observed higher the alcohol content better is the wine quality**

#**Split data as predictors and target**

In [ ]:
#Last 13 columns as predictors
X = data.iloc[ :, 1:14]
X.head(2)

In [ ]:
#First column as target
y = data.iloc[ : , 0]
y.head()

In [ ]:
y = y.map({'white':1, 'red' : 0})
y.dtype

In [ ]:
#Divide dataset into Training data and validation data
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.35,
                                                    shuffle = True
                                                    )

#**Hyperparameter tuning and Pipelining using XGBoost **

In [ ]:
xgb_list = [('ss',ss()),         #Scaling parameters
            ('pca',PCA()),       #Instantiate PCA
            ('xgb',XGBClassifier(silent = False,  #Instantiate XGB classifier with 2 cpu threads 
                                  n_jobs=2))]

In [ ]:
#Instantiate Pipeline object
xgb_pipeline = Pipeline(xgb_list)


In [ ]:
#A parameter grid for grid search
parameter_gs = {'xgb__learning_rate':  [0.04, 0.07],
              'xgb__n_estimators':   [150,  200],
              'xgb__max_depth':      [3,5],
              'pca__n_components' : [7,12]
              }

##**Grid Search**

In [ ]:
grid_search = GridSearchCV(xgb_pipeline,            # XGB pipeline object
                   parameter_gs,         # 2*2*2*2 parameter grid
                   n_jobs = 2,         # No. of parallel cpu threads
                   cv =2 ,             # No. of folds
                   verbose =2,      
                   scoring = ['accuracy', 'roc_auc'],  # Performance metrics
                   refit = 'roc_auc'   # Refitting final model those which maximise auc
                   )

In [ ]:
#Fitting data to Pipeline
start = time.time()
grid_search.fit(X_train, y_train)   
end = time.time()
(end - start)/60 

In [ ]:
f"Best score: {grid_search.best_score_} "

In [ ]:
f"Best parameter set {grid_search.best_params_}"

In [ ]:
plt.bar(grid_search.best_params_.keys(), grid_search.best_params_.values(), color='g')
plt.xticks(rotation=10)

In [ ]:
y_pred = grid_search.predict(X_test)
y_pred

In [ ]:
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
f"Accuracy: {accuracy * 100.0}"

##**Random Search**

In [ ]:
#A parameter set for random search
parameter_rs = {'xgb__learning_rate':  uniform(0, 1),
              'xgb__n_estimators':   range(60,120),
              'xgb__max_depth':      range(4,7),
              'pca__n_components' : range(5,7)}

In [ ]:
random_search = RandomizedSearchCV(xgb_pipeline,
                        param_distributions=parameter_rs,
                        scoring= ['roc_auc', 'accuracy'],
                        n_iter=15,          # Max combination of
                        verbose = 3,
                        refit = 'roc_auc',
                        n_jobs = 2,          # No. of parallel cpu threads
                        cv = 2               # No of folds.
                        )

In [ ]:
#Fitting data to Pipeline
start = time.time()
random_search.fit(X_train, y_train)
end = time.time()
(end - start)/60

In [ ]:
f"Best score: {random_search.best_score_} "

In [ ]:
f"Best parameter set: {random_search.best_params_} "

In [ ]:
plt.bar(random_search.best_params_.keys(), random_search.best_params_.values(), color='g')
plt.xticks(rotation=10)

In [ ]:
y_pred = random_search.predict(X_test)
y_pred

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
f"Accuracy: {accuracy * 100.0}"

##**Bayesian Optimization**

In [ ]:
parameter_bo = {
           'learning_rate':  (0, 1),            
           'n_estimators':   (60,120),         
           'max_depth':      (4,7),            
           'n_components' :  (5,7)
            }

In [ ]:

def xg_eval(learning_rate,n_estimators, max_depth,n_components):
    #Make Pipeling for BO
    pipe_xg1 = make_pipeline (ss(),
                              PCA(n_components=int(round(n_components))),
                              XGBClassifier(
                                           silent = False,
                                           n_jobs=2,
                                           learning_rate=learning_rate,
                                           max_depth=int(round(max_depth)),
                                           n_estimators=int(round(n_estimators))
                                           )
                             )
    #Fitting into pipeline 
    cv_result = cross_val_score(estimator = pipe_xg1,
                                X= X_train,
                                y = y_train,
                                cv = 2,
                                n_jobs = 2,
                                scoring = 'f1'
                                ).mean()             # taking mean of all results

    return cv_result       #Returning final mean of all results of cross val score

In [ ]:
bayesian_opt = BayesianOptimization(
                             xg_eval,     
                             parameter_bo   
                             )

In [ ]:
start = time.time()

In [ ]:
bayesian_opt.maximize(init_points=5,
               n_iter=15,        
               )

In [ ]:
f"Best parameter set: {bayesian_opt.max} "

In [ ]:
bayesian_opt.max.values()

In [ ]:
for features in bayesian_opt.max.values(): 
    print(features)

In [ ]:
features

In [ ]:
plt.bar(features.keys(), features.values(), color='g')
plt.xticks(rotation=10)

#**Fitting parameters into our model and Feature Importance**

In [ ]:
#Model with parameters of grid search
model_gs = XGBClassifier(
                    learning_rate = grid_search.best_params_['xgb__learning_rate'],
                    max_depth = grid_search.best_params_['xgb__max_depth'],
                    n_estimators=grid_search.best_params_['xgb__n_estimators']
                    )

#Model with parameters of random search
model_rs = XGBClassifier(
                    learning_rate = random_search.best_params_['xgb__learning_rate'],
                    max_depth = random_search.best_params_['xgb__max_depth'],
                    n_estimators=random_search.best_params_['xgb__n_estimators']
                    )

#Model with parameters of bayesian optimization
model_bo = XGBClassifier(
                    learning_rate = int(features['learning_rate']),
                    max_depth = int(features['max_depth']),
                    n_estimators=int(features['n_estimators'])
                    )

In [ ]:
start = time.time()
model_gs.fit(X_train, y_train)
model_rs.fit(X_train, y_train)
model_bo.fit(X_train, y_train)

In [ ]:
y_pred_gs = model_gs.predict(X_test)
y_pred_rs = model_rs.predict(X_test)
y_pred_bo = model_bo.predict(X_test)

In [ ]:
accuracy_gs = accuracy_score(y_test, y_pred_gs)
accuracy_rs = accuracy_score(y_test, y_pred_rs)
accuracy_bo = accuracy_score(y_test, y_pred_gs)

In [ ]:
print("Grid search Accuracy: "+str(accuracy_gs))
print("Grid search Accuracy: "+str(accuracy_rs))
print("Bayesian Optimization Accuracy: "+str(accuracy_bo))

In [ ]:
model_gs.feature_importances_
model_rs.feature_importances_
model_bo.feature_importances_
plot_importance(model_gs)
plot_importance(model_rs)
plot_importance(model_bo)
plt.show()